In [ ]:

# Save figures in specific place

knitr::opts_chunk$set(autodep        = TRUE,
                      cache          = FALSE,
                      cache.comments = TRUE,

                      # Save figures as pdf ?
                      dev = c("png", "pdf"),

                      # Include code?
                      echo           = TRUE,
                      warning = FALSE,

                      error          = FALSE,
                      fig.align      = "center",

                      # Path where figures are going to be store pdf single
                      # figures
                      fig.path       = paste0("./figures", "/"),
                      #fig.width      = 11,
                      #fig.height     = 7,
                      message        = FALSE)

# set to two decimal
options(scipen = 1, digits = 4)

In [ ]:
library(janitor)
library(ggpubr)
library(knitr)
library(ggsci)
library(tidyverse)
library(here)
library(nlme)
library(performance)
library(reactablefmtr)
library(emmeans)
library(modelr)
library(car)

In [ ]:
# Set knit directory
setwd(here::here())
knitr::opts_knit$set(root.dir = setwd(here::here()))
getwd()

# Load functions and data

In [ ]:
# Load all joined dataset
source("./scripts/code_join_data_full_dataset.R")
source("./scripts/code_clean_data_soil_moisture.R")

In [ ]:
# Load custom made functions
source("./R/functions_models.R")
source("./R/function_plots.R")
source("./R/function_validation_plots.R")
source("./R/function_for_inference_emmeans_and_percentage_diff.R")

# Soil moisture figures

In [ ]:
data_soil_moisture_cleaned  %>% 
	group_by(treatment,nfixer, date_day_month,sm_measured ) %>%

	summarise(mean_sm = mean(soil_moisture),
			  sd_sm   = sd(soil_moisture)) %>%

        arrange(nfixer) %>%
	arrange(date_day_month) %>%
	kable()

## Boxplots

In [ ]:
ggplot(data = data_soil_moisture_cleaned, 
        aes(x = date_day_month, y = soil_moisture, fill = treatment)) +
	
	#Adjust boxplots side by side
	geom_boxplot(position = position_dodge(width=.8)) +
	theme_bw() +

        facet_wrap(~ sm_measured, scales = "free_y") +
		#Define colors
	#scale_colour_manual(values = c("#F0E442","#0072B2"))+
	xlab("Date") + ylab("Mean Pot Soil Moisture %") +

	#Delete lines around facet labels
	theme(strip.background = element_rect( color="white", fill="white"),

                # Legend position and size
                legend.position = "bottom",
                legend.title = element_text(size = 18),
                legend.text = element_text(size = 18)) +

	guides(col = guide_legend(ncol = 1, title.position = "left",
                                title.hjust = .59,
                                title = "Soil Moisture Measuared:")) +
        scale_fill_manual(values = c("#DDAA33","#117733",
                                            "#BB5566","#004488")) +
        theme_classic() +
        theme(legend.position = "none") +
        theme(axis.text = element_text(size = 21),
                axis.title = element_text(size = 21, face = "bold")) 

## Mean values + Standard errors

In [ ]:
data_soil_moisture_cleaned  %>%
        group_by(date_day_month, sm_measured, treatment) %>%
        summarise(mean_soil_moisture = mean(soil_moisture),

                se_soil_moisture = (sd(soil_moisture) / sqrt(n())))  %>%

        mutate(difference = ((mean_soil_moisture - first(mean_soil_moisture)))) %>%
        filter(sm_measured == "After applying the treatments" &
                treatment %in% c("Ambient Rain plus Water",
                                "Ambient Rain plus Nutrients and Water")) %>%
        kable()

In [ ]:
data_soil_moisture_cleaned  %>%
        group_by(date_day_month, sm_measured, treatment) %>%
        summarise(mean_soil_moisture = mean(soil_moisture),

                se_soil_moisture = (sd(soil_moisture) / sqrt(n()))) %>% 
                
        ggplot(data = ., aes(x = date_day_month, y = mean_soil_moisture,
                            color = treatment)) +
            geom_point(position = position_dodge(width = .9), size = 3.5) +
            
            facet_wrap(~sm_measured) +

            scale_color_manual(values = c("#DDAA33","#117733",
                                            "#BB5566","#004488")) +

            xlab("Date") + ylab("Mean Pot Soil Moisture %") +
            theme_bw() +

            # Add Standard error bars
            geom_errorbar(aes(ymin = mean_soil_moisture - se_soil_moisture,
                              ymax = mean_soil_moisture + se_soil_moisture),
                            width = .2,
                            position = position_dodge(width = .9)) +


            #Delete lines around facet labels
	        theme(

                    #Font size
                    axis.text = element_text(size = 18),
                    axis.title = element_text(size = 18, face = "bold"),
                    strip.text.x = element_text(size = 18, face = "bold"),

                    # Legend position and size
                    legend.position = "none",
                    legend.title = element_text(size = 18),
                    legend.text = element_text(size = 18)) +
	        
                guides(col = guide_legend(ncol = 1, title.position = "top",
                            title.hjust = .59))

# Plant's mass fractions figures

In [ ]:
data_mass_fractions <-
    data_for_models %>%
    tibble::rownames_to_column("id") %>%
    select(id,spcode, treatment, nfixer, init_height)

## Model fitting

In [ ]:
data_for_models <-
  data_for_models %>%

  # Select variables for analysis
  dplyr::select(c(spcode, treatment, nfixer, init_height, rmf, smf, lmf))

In [ ]:
models_mass_fractions <- list()

In [ ]:
model_rmf <- lme(rmf ~ nfixer*treatment + init_height,
                        random =  ~1|spcode,
                        data = data_for_models)

model_rmf <- list(model_rmf)

names(model_rmf) <- "rmf"

# Append log models to model list
models_mass_fractions <- append(model_rmf, models_mass_fractions)

In [ ]:
# Log model

## Root-Shoot ratio
model_smf_log <- lme(log(smf) ~ nfixer*treatment + init_height,
                        random = ~1|spcode,
                        data = data_for_models)

log_model_smf <- list(model_smf_log)

names(log_model_smf) <- "smf_log"

# Append log models to model list
models_mass_fractions <- append(log_model_smf, models_mass_fractions)

In [ ]:
model_lmf <- lme(lmf ~ nfixer*treatment + init_height,
                        random =  ~1|spcode,
                        data = data_for_models)

model_lmf <- list(model_lmf)

names(model_lmf) <- "lmf"

# Append log models to model list
models_mass_fractions <- append(model_lmf, models_mass_fractions)

In [ ]:
names(models_mass_fractions)

## Model Assumptions

### Root Mass fractions

In [ ]:
validation_plots(models_mass_fractions$rmf,
                data = data_for_models,
                group = "spcode")

### Stem Mass fractions

In [ ]:
validation_plots(models_mass_fractions$smf_log,
                data = data_for_models,
                group = "spcode")

### Leaf Mass fractions

In [ ]:
validation_plots(models_mass_fractions$lmf,
                data = data_for_models,
                group = "spcode")

## Model inference

In [ ]:
## r2 models
models_mass_fractions %>%
    map(., r2) %>%
    unlist() %>%
    kable()

### Anova tables

In [ ]:
map(models_mass_fractions, ~anova.lme(.x, type = "marginal"))

## Post-Hoc: Tukey's test

### Root Mass fractions

In [ ]:
as_tibble(emmeans(models_mass_fractions$rmf,
        pairwise ~ nfixer*treatment,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

### Stem Mass fractions

In [ ]:
as_tibble(emmeans(models_mass_fractions$smf_log,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

### Leaf Mass fractions

In [ ]:
as_tibble(emmeans(models_mass_fractions$lmf,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

## Treatment effects

### Root Mass fractions

In [ ]:
emmeans_table_tidy(models_mass_fractions$rmf,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

### Stem Mass fractions

In [ ]:
emmeans_table_tidy(models_mass_fractions$smf_log,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

### Leaf Mass fractions

In [ ]:
emmeans_table_tidy(models_mass_fractions$lmf,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

## Boxplots Mass fractions

In [ ]:
# Step done for getting predictions from models for Q1
data_for_predictions <-
    data_for_models %>%

        rownames_to_column("id") %>%

        # Remove unused variables
        dplyr::select(id, spcode, treatment, nfixer, init_height)

In [ ]:
# Get predictions
string <- c("models_mass_fractions")

data_pred_mass_fractions <-

        # Get models prediction
        modelr::gather_predictions(data_for_predictions,

                           # Return predictions
                            models_mass_fractions$rmf,
                            models_mass_fractions$smf_log,
                            models_mass_fractions$lmf) %>%

        pivot_wider(names_from = model, values_from = pred) %>%
            rename_all(funs(

                # rename columns
                stringr::str_to_lower(.) %>%
                stringr::str_replace(., c(string),"pred_") %>%

                # Remove dollar sing
                gsub("\\$", "", .)
                )) %>%

        # Back transform log variables
        mutate(pred_smf = exp(pred_smf_log)) %>%

        # Remove log predictions and init height
        dplyr::select(-c(init_height, pred_smf_log))

In [ ]:
vars_mass_fractions <-
  crossing(

    # Get all numeric variables to plot (all y)
    as_tibble(t(combn(dplyr::select(data_pred_mass_fractions,
                                    where(is.numeric)) %>% names, 1))),

    # Select factor variables to plot
    x_axis_var = dplyr::select(data_pred_mass_fractions, nfixer) %>%  names,
    group_var = dplyr::select(data_pred_mass_fractions, treatment) %>%  names)

In [ ]:
vars_mass_fractions %>%
      # Gererate plots
      pmap( ~ boxplot_plot_pmap(data = data_pred_mass_fractions,
                                y = !!sym(..1), x = !!sym(..2),
                                fill = !!sym(..3))) 
